In [1]:
import os
from dotenv import load_dotenv,find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain_openai import ChatOpenAI 

chatModel = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me a curious fact about {politician}")

chain = prompt | chatModel | StrOutputParser()

In [6]:
chain.invoke({"politician":"Chandra babu naidu"})

'One curious fact about Chandra Babu Naidu is that he is known for his innovative use of technology in governance. He was one of the first Indian politicians to embrace the use of IT and social media in his administration, earning him the nickname of "tech-savvy politician." Naidu was instrumental in transforming the city of Hyderabad into a major IT hub during his tenure as Chief Minister of Andhra Pradesh.'

In [10]:
for s in chain.stream({"politician":"Nara Lokesh"}):
    print(s,end="",flush=True)

Nara Lokesh, an Indian politician and businessman, holds a Bachelor's degree in Management Information Systems from Carnegie Mellon University, USA, and a Master's degree in Business Administration from Stanford University, USA.

In [11]:
for s in chain.batch([{"politician":"Nara Lokesh"},{"politician":"Narendra Modi"}]):
    print(s,end="",flush=True)

Nara Lokesh, an Indian politician and businessman, is known for being the son of Nara Chandrababu Naidu, the former Chief Minister of the Indian state of Andhra Pradesh. Despite being born into a political family, Lokesh holds a degree in Bachelor of Science in Management Information Systems from Carnegie Mellon University in the United States and has worked in the corporate sector before entering politics. This unique combination of a political background and a strong educational and professional background sets Lokesh apart from many other politicians in India.Narendra Modi is known for his strict vegetarian diet and teetotaler lifestyle, which is uncommon for many Indian politicians known for their lavish feasts and indulgence in alcohol.

### Legacy Chain

In [7]:
from langchain.chains import LLMChain 

prompt = ChatPromptTemplate.from_template("Tell me a curious fact about {soccer_player}")

traditional_chain = LLMChain(
    llm = chatModel,
    prompt = prompt
)

traditional_chain.predict(soccer_player="Maradona")

C:\Users\DELL\AppData\Local\Temp\ipykernel_17280\39452126.py:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  traditional_chain = LLMChain(


'One curious fact about Maradona is that he was once suspended from playing football for 15 months after testing positive for cocaine in 1991.'

### Built In Runnables in LCEL

- RunnablePassThrough
- RunnableLambda
- RunnableParallel
  - ItemGetter

In [12]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model = "gpt-3.5-turbo-0125")

##### RunnablePassThrough

- It does not do anything to the input data.
- Let's see it in a very simple example : A chain with just RunnablePassthrough() will output the original input without any modification.

In [14]:
from langchain_core.runnables import RunnablePassthrough

run_chain = RunnablePassthrough()

In [16]:
run_chain.invoke("sai krishna")

'sai krishna'

### RunnableLambda

- To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.
- Lets define a very simple function to create Russian lastname.


In [17]:
def russian_lastname(name:str)->str:
    return f"{name}ovich"


In [18]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

chain.invoke("sai")

'saiovich'

### RunnableParallel

- We use RunnableParallel() for running tasks in parallel
- This is probably the most important and most useful Runnable from Langchain
- In the following chain, RunnableParallel is going to run these two tasks in parallel:
   - Operation_a will use RunnablePassthrough
   - operation_b will use RunnableLambda with the russian_lastname function.

   

In [19]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

chain.invoke("Bappy")

{'operation_a': 'Bappy', 'operation_b': 'Bappyovich'}

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [21]:
prompt = ChatPromptTemplate.from_template("Tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()


In [22]:
def russian_lastname_from_dictionary(person):
    return person["name"]+"ovich"



In [24]:
chain = RunnableParallel(
    {
        "operation_a":RunnablePassthrough(),
        "soccer_player":RunnableLambda(russian_lastname_from_dictionary),
        "operation_c" : RunnablePassthrough(),
    }
) | prompt | model | output_parser

chain.invoke({
    "name1":"Jordam",
    "name":"krishna"
})

'One curious fact about krishnaovich is that it is a surname that is of Russian origin and is derived from the given name "Krishna," which is a Hindu deity. This unique combination of Russian and Hindu influences makes it a distinctive and intriguing surname.'

### Advanced Model for Runnableparallel

In [25]:
from langchain_community.vectorstores import FAISS 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough,RunnableParallel
from langchain_openai import ChatOpenAI,OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["DSwithbappy focuses on providing content on Data Science, AI, ML, DL, CV , NLP, Python in English"],embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context: 
{context}

Question : {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-3.5-turbo")

retrieval_chain = (
    RunnableParallel({"context":retriever,"question": RunnablePassthrough()})
    | prompt
    | model 
    | StrOutputParser()
)

retrieval_chain.invoke("what is DSwithBappy?")

'DSwithBappy is a platform that provides content on Data Science, AI, Machine Learning, Deep Learning, Computer Vision, Natural Language Processing, and Python in English.'

In [28]:
from langchain_community.vectorstores import FAISS 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough,RunnableParallel
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from operator import itemgetter

vectorstore = FAISS.from_texts(
    ["DSwithbappy focuses on providing content on Data Science, AI, ML, DL, CV , NLP, Python in English"],embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context: 
{context}

Question : {question}

Answer in the following language : {language}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-3.5-turbo")

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question" : itemgetter("question"),
        "language": itemgetter("language")
    }
    | prompt
    | model 
    | StrOutputParser()
)

chain.invoke({"question": "what is DSwithBappy?","language":"Hindi"})

'DSwithBappy डेटा साइंस, एआई, एमएल, डीएल, सीवी, एनएलपी, पायथन पर सामग्री प्रदान करने पर ध्यान केंद्रित करता है।'

### Built-in Functions in Runnables

In [29]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")


In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [31]:
prompt = ChatPromptTemplate.from_template("Tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"soccer_player":"Ronaldo"})

'One curious fact about Cristiano Ronaldo is that he holds the record for the most goals scored in a single UEFA Champions League season, with 17 goals during the 2013-2014 season while playing for Real Madrid.'

### use of .bind() to add arguments to a Runnable in a LCEL Chain
- For example , we can add an argument to stop the model response when it reaches the word "Ronaldo":

In [32]:
func_chain = prompt | model.bind(stop=["Ronaldo"]) | output_parser

func_chain.invoke({"soccer_player":"Ronaldo"})

'One curious fact about Cristiano '

### Combining Chains in LCEL

In [37]:
prompt = ChatPromptTemplate.from_template("Tell me a sentence about politician {politician}")

output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke("hitler")

'Adolf Hitler was a dictator and politician who led Germany into World War II and orchestrated the Holocaust.'

### Combined Chain

In [36]:
historian_prompt = ChatPromptTemplate.from_template("Was {politician} positive for humanity?")

composed_chain = {"politician":chain} | historian_prompt | model | StrOutputParser()

composed_chain.invoke({"politician": "hitler"})

'No, Adolf Hitler was not a positive figure for humanity. He was responsible for the genocide of six million Jews during the Holocaust, as well as the deaths of millions of other civilians during World War II. His actions led to immense suffering, destruction, and loss of life, making him one of the most notorious and cruel leaders in history.'

### Example-2

In [40]:
prompt1 = ChatPromptTemplate.from_template("What is the country {politician} is from?")

prompt2 = ChatPromptTemplate.from_template(
    "What continent is the country {country} in? respond in {language}"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"country":chain1,"language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"politician":"Nara Lokesh","language":"telugu"})

'నారా లోకేష్ భారత దేశం నుండి వచ్చిన రాజకీయ నాయకుడు. ఆయన ఆంధ్ర ప్రదేశ్ రాష్ట్రంలో తెలుగు దేశం పార్టీ సభ్యుడు. భూమిమండలం ఆది తెలుగు దేశం.'

### LCEL Chain at work in Typical RAG App

In [44]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

rag_chain.invoke("what are AI Agents")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
c:\Users\DELL\anaconda3\envs\llmapp\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
c:\Users\DELL\anaconda3\envs\llmapp\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
c:\Users\DELL\anaconda3\envs\llmapp\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessin

'AI Agents are virtual characters controlled by LLM-powered agents that interact in simulated environments. They are designed to create believable human-like behavior for interactive applications. These agents combine LLM with memory, planning, and reflection mechanisms to interact with each other based on past experiences.'